# Выбор локации для скважины

<b>Описание проекта</b>  

Мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой Bootstrap.

Шаги для выбора локации:

    В избранном регионе ищут месторождения, для каждого определяют значения признаков;
    Строят модель и оценивают объём запасов;
    Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
    Прибыль равна суммарной прибыли отобранных месторождений.

Описание данных

Признаки:
id — уникальный идентификатор скважины;
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
Целевой признак:
product — объём запасов в скважине (тыс. баррелей).

Условия задачи:

    Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
    При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
    Бюджет на разработку скважин в регионе — 10 млрд рублей.
    При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
    После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
    

План проведения работы

1. Загрузим и подготовим данные. Поясним порядок действий.  
2. Обучим и проверим модель для каждого региона:  
2.1. Разобьём данные на обучающую и валидационную выборки в соотношении 75:25.  
2.2. Обучим модель и сделаем предсказания на валидационной выборке.  
2.3. Сохраним предсказания и правильные ответы на валидационной выборке.  
2.4. Напечатаем на экране средний запас предсказанного сырья и RMSE модели.  
2.5. Проанализируем результаты.  
3. Подготовимся к расчёту прибыли:  
3.1. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.  
3.2. Напишем выводы по этапу подготовки расчёта прибыли.  
4. Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:  
4.1. Выберем скважины с максимальными значениями предсказаний.  
4.2. Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.  
4.3. Рассчитаем прибыль для полученного объёма сырья.  
5. Посчитаем риски и прибыль для каждого региона:  
5.1. Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.  
5.2. Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.  
5.3. Напишем выводы: предложим регион для разработки скважин и обоснуем выбор.  

    



## Загрузка и подготовка данных

In [23]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
from scipy import stats as st

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [24]:
# Сохраним константы в переменных
# Бюджет на разработку скважин в регионе, тыс.руб
BUDGET = 10000000 
# Доход с каждой единицы продукта, тыс.руб
INCOME_ONE_BARREL = 450
# Количество скважин для разведки региона
TOTAL_WELL = 500
# Количество скважин для разработки
BEST_WELL = 200

In [25]:
# Читаем данные из csv-файлов и сохраним в переменные
data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv') 

In [26]:
# Напишем функцию для изучения общей информации о полученных датафреймах 
def review(data): 
    info = display(data.head(), data.info(), data.describe(), 
    data.corr(numeric_only = True)
    .style.background_gradient(cmap='PRGn'))                  
    print('Количество дупликатов:', data.duplicated().sum()) 

In [27]:
# Изучим данные 1 региона
review(data_0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


Количество дупликатов: 0


In [28]:
# Изучим данные 2 региона
review(data_1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


Количество дупликатов: 0


In [29]:
# Изучим данные 3 региона
review(data_2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Количество дупликатов: 0


Признаки 'id' в каждой таблице - идентификаторы скважины - не несут ценности для обучения будущей модели, поэтому их можно удалить. 

In [30]:
data_0.drop(columns='id', inplace=True)
data_1.drop(columns='id', inplace=True)
data_2.drop(columns='id', inplace=True)

В каждой таблице по 5 столбцов и 100000 строк. Пропусков и явных дупликатов нет. Все данные выглядят корректно. Нет явно выбивающихся значений. Признаки 'id' - идентификаторы скважины - не несут ценности для обучения будущей модели, поэтому мы их удалили. Имеется высокая зависимость показателя объёма запасов нефти от признака f2 для 1 и 3 региона. В регионе 2 мы видим прямую и линейную зависимость целевого показателя 'product' от f2, такая зависмость можеть дать нам высокие оценки предсказаний для нашей модели. 

## Обучение и проверка модели

### Разобьем данные на обучающую и валидационную выборки в соотношении 75:25.

Подготовим данные для 1 региона

In [31]:
# Запишем признаки в переменные
target_0=data_0['product']
features_0=data_0.drop(['product'], axis=1)

# Разобьём данные на обучающую и валидационную выборки
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, 
                                                                                      test_size=0.25, random_state=42)

# Выведем на экран размеры этих выборок
print('Размер валидационной выборки: {:.0%}'.format(target_valid_0.shape[0]/target_0.shape[0]))
print('Размер тренировочной выборки: {:.0%}'.format(target_train_0.shape[0]/target_0.shape[0]))

Размер валидационной выборки: 25%
Размер тренировочной выборки: 75%


Подготовим данные для 2 региона

In [32]:
# Запишем признаки в переменные
target_1=data_1['product']
features_1=data_1.drop(['product'], axis=1)

# Разобьём данные на обучающую и валидационную выборки
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1,
                                                                                      test_size=0.25, random_state=42)

# Выведем на экран размеры этих выборок
print('Размер валидационной выборки: {:.0%}'.format(target_valid_1.shape[0]/target_1.shape[0]))
print('Размер тренировочной выборки: {:.0%}'.format(target_train_1.shape[0]/target_1.shape[0]))

Размер валидационной выборки: 25%
Размер тренировочной выборки: 75%


Подготовим данные для 3 региона

In [33]:
# Запишем признаки в переменные
target_2=data_2['product']
features_2=data_2.drop(['product'], axis=1)

# Разобьём данные на обучающую и валидационную выборки
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2,
                                                                                      test_size=0.25, random_state=42)

# Выведем на экран размеры этих выборок
print('Размер валидационной выборки: {:.0%}'.format(target_valid_2.shape[0]/target_2.shape[0]))
print('Размер тренировочной выборки: {:.0%}'.format(target_train_2.shape[0]/target_2.shape[0]))

Размер валидационной выборки: 25%
Размер тренировочной выборки: 75%


### Обучим модель и сделаем предсказания на валидационной выборке.

In [34]:
# Обучим модель для 1 региона
model_0=LinearRegression()
model_0.fit(features_train_0,target_train_0)
predictions_0=model_0.predict(features_valid_0)

In [35]:
# Обучим модель для 2 региона
model_1=LinearRegression()
model_1.fit(features_train_1,target_train_1)
predictions_1=model_1.predict(features_valid_1)

In [36]:
# Обучим модель для 3 региона
model_2=LinearRegression()
model_2.fit(features_train_2,target_train_2)
predictions_2=model_2.predict(features_valid_2)

### Сохраним предсказания и правильные ответы на валидационной выборке

In [37]:
print('1 регион:\n- предсказания модели: predict_0\n- правильные ответы: target_valid_0\n')

print('2 регион:\n- предсказания модели: predict_1\n- правильные ответы: target_valid_1\n')

print('3 регион:\n- предсказания модели: predict_2\n- правильные ответы: target_valid_2')

1 регион:
- предсказания модели: predict_0
- правильные ответы: target_valid_0

2 регион:
- предсказания модели: predict_1
- правильные ответы: target_valid_1

3 регион:
- предсказания модели: predict_2
- правильные ответы: target_valid_2


### Напечатаем на экране средний запас предсказанного сырья и RMSE модели.

In [38]:
av_table = pd.DataFrame(index=['средний запас предсказанного сырья','RMSE модели'], columns=['1 регион','2 регион','3 регион'])
av_table['1 регион'] = predictions_0.mean(), mean_squared_error(target_valid_0, predictions_0)**0.5
av_table['2 регион'] = predictions_1.mean(), mean_squared_error(target_valid_1, predictions_1)**0.5
av_table['3 регион'] = predictions_2.mean(), mean_squared_error(target_valid_2, predictions_2)**0.5
av_table

,1 регион,2 регион,3 регион
средний запас предсказанного сырья,92.3988,68.712878,94.771024
RMSE модели,37.7566,0.890280,40.145872


В 1 и 3 регионе средние значение объема в месторождении примерно одинаковые, при этом модели имеют большую погрешность.
В 2 регионе средние запасы нефти в месторождениях ниже больше чем в 2 раза, но и погрешность модели значительно ниже. 

## Подготовка к расчёту прибыли

### Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [39]:
# Посчитаем бюджет на разработку одной скважины
budget_for_well=(BUDGET/BEST_WELL/1000)
print('Бюджет на разработку одной скважины:', budget_for_well,'млн.руб')

Бюджет на разработку одной скважины: 50.0 млн.руб


In [40]:
vol_barrel = budget_for_well / INCOME_ONE_BARREL * 1000
print('Достаточный объём сырья для безубыточной разработки новой скважины не менее: %.2f'%vol_barrel,'тыс.баррелей')

Достаточный объём сырья для безубыточной разработки новой скважины не менее: 111.11 тыс.баррелей


In [41]:
# Посчитаем средний запас сырья для каждого региона
print('Cредний запас сырья в 1 регионе: %.2f тыс.баррелей'%data_0['product'].mean())
print('Cредний запас сырья в 2 регионе: %.2f тыс.баррелей'%data_1['product'].mean())
print('Cредний запас сырья в 3 регионе: %.2f тыс.баррелей'%data_2['product'].mean())

Cредний запас сырья в 1 регионе: 92.50 тыс.баррелей
Cредний запас сырья в 2 регионе: 68.83 тыс.баррелей
Cредний запас сырья в 3 регионе: 95.00 тыс.баррелей


Среднее количество запасов сырья по всем трём регионам недостаточное для безубыточной разработки новой скважины.

## Напишем функцию для расчёта прибыли по лучшим скважинам и предсказаниям модели

In [42]:
def total_profit_one(predictions, target, count):
    # Отсортируем предсказания модели для скважин по убыванию 
    well_sorted = pd.Series(predictions, index=target.index).sort_values(ascending=False)
    # Просуммируем показатели самых прибыльных скважин
    max_sorted = target[well_sorted.index][:count].sum()
    # Рассчитаем прибыль для полученного объёма сырья
    revenue = (max_sorted * INCOME_ONE_BARREL) - BUDGET
    print('Суммарный запас сырья в 200 лучших скважинах составит: %.2f тыс.баррелей'%max_sorted)
    print('Прибыль для полученного объёма сырья составит %.2f млн.рублей'%(revenue/1000))

In [43]:
# Рассчет для 1 региона
total_profit_one(predictions_0, target_valid_0, BEST_WELL)

Суммарный запас сырья в 200 лучших скважинах составит: 29686.98 тыс.баррелей
Прибыль для полученного объёма сырья составит 3359.14 млн.рублей


In [44]:
# Рассчет для 2 региона
total_profit_one(predictions_1, target_valid_1, BEST_WELL)

Суммарный запас сырья в 200 лучших скважинах составит: 27589.08 тыс.баррелей
Прибыль для полученного объёма сырья составит 2415.09 млн.рублей


In [45]:
# Рассчет для 3 региона
total_profit_one(predictions_2, target_valid_2, BEST_WELL)

Суммарный запас сырья в 200 лучших скважинах составит: 27996.83 тыс.баррелей
Прибыль для полученного объёма сырья составит 2598.57 млн.рублей


## Расчёт прибыли и рисков 

### Сделаем бутстреп для каждого региона, посмотрим на прибыль и 95% доверительный интервал.

In [46]:
def total_profit_two(predictions, target, count):
    well_sorted = pd.Series(predictions).sort_values(ascending=False)
    max_well = target[well_sorted.index][:count].sum()   
    revenue = (max_well * INCOME_ONE_BARREL) - BUDGET
    return revenue 

def bootstap_1000(target,predictions):
    values = []
    count=0
    state = np.random.RandomState(42)
    predictions = pd.Series(predictions, index=target.index)
    
    for i in range(1000):
        target_subsample = target.sample(n=TOTAL_WELL,replace=True, random_state=state)
  
        probs_subsample = predictions[target_subsample.index]
        values.append(total_profit_two(probs_subsample,target_subsample, BEST_WELL))  
    for i in values:
        if i < 0:
            count +=1
            
    values = pd.Series(values)
    mean = values.mean()
    confidence_interval = st.norm.interval(0.95, len(values)-1, values.mean())    
    
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    
    print("Средняя прибыль в регионе составит {:.2f} млн.руб".format(mean/1000))
    print('С вероятностью 95% прибыль в данном регионе будет составлять: от {:.2f} млн.руб до {:.2f} млн.руб'.format((lower/1000),(higher/1000)))
    print("95%-ый доверительный интервал: от {:.2f} млн.руб до {:.2f} млн.руб". format((confidence_interval[0]/1000),(confidence_interval[1]/1000)))
    print ('Процент вероятности убытка в данном регионе составит - {:.1%}'.format(count/1000))


In [47]:
# Проведем расчеты для 1 региона
bootstap_1000(target_valid_0, predictions_0)

Средняя прибыль в регионе составит 427.85 млн.руб
С вероятностью 95% прибыль в данном регионе будет составлять: от -97.25 млн.руб до 954.22 млн.руб
95%-ый доверительный интервал: от -837.57 млн.руб до 839.56 млн.руб
Процент вероятности убытка в данном регионе составит - 5.5%


In [48]:
# Проведем расчеты для 2 региона
bootstap_1000(target_valid_1, predictions_1)

Средняя прибыль в регионе составит 511.36 млн.руб
С вероятностью 95% прибыль в данном регионе будет составлять: от 98.87 млн.руб до 940.72 млн.руб
95%-ый доверительный интервал: от -1001.25 млн.руб до 1003.25 млн.руб
Процент вероятности убытка в данном регионе составит - 0.9%


In [49]:
# Проведем расчеты для 3 региона
bootstap_1000(target_valid_2, predictions_2)

Средняя прибыль в регионе составит 402.58 млн.руб
С вероятностью 95% прибыль в данном регионе будет составлять: от -137.16 млн.руб до 929.89 млн.руб
95%-ый доверительный интервал: от -788.03 млн.руб до 790.03 млн.руб
Процент вероятности убытка в данном регионе составит - 7.4%


По условиям задачи нам надо оставить регионы с вероятность убытков меньше 2.5% и выбрать среди них самы прибыльный. Для заданных условий подходит только один регион - регион 2: процент вероятности убытка в данном регионе составит - 0.9% и средняя прибыль в регионе составит 511.36 млн.руб. 

## Общий вывод

Было проведено исследование, чтобы спрогнозировать регион, где добыча нефти принесёт наибольшую прибыль. Надо было проанализировать возможную прибыль и риски техникой Bootstrap с разделением каждого предсказанного значения прибыли по регионам на 1000 выборок. Далее оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирать регион с наибольшей средней прибылью.  
В ходе работы была выполнена предобработка данных.  
Признаки 'id' удалили, это идентификаторы скважины - не несут ценности для обучения будущей модели.
Пропусков и явных дупликатов не нашлось. Все данные выглядят корректно. Нет явно выбивающихся значений. Выявили высокую зависимость показателей объёма запасов нефти от признака f2 для 1 и 3 региона. В регионе 2 мы видим прямую и линейную зависимость целевого показателя 'product' от f2.  
Разобили данные на обучающую и валидационную выборки в соотношении 75:25. Обучили модель и выполнили предсказания на валидационной выборке при помощи линейной регрессии.    
В 1 и 3 регионе средние значение объема в месторождении примерно одинаковые, при этом модели имеют большую погрешность. В 2 регионе средние запасы нефти в месторождениях ниже больше чем в 2 раза, но и погрешность модели значительно ниже.  
Определили, что среднее количество запасов сырья по всем трём регионам недостаточное даже для безубыточной разработки новой скважины.  
Применили технологию bootstrap. По результатам полученных данных мы определили доверительный интревал получения прибыли в 95%, ограничив вероятность убытка величиной менее 2,5%. И на основе этих данных смогли выбрать более перспективный регион для разработки 200 скважин.  
Т.о., мы можем порекоммендовать для дальнейшей разработки регион 2: процент вероятности убытка в данном регионе составит - 0.9% и средняя прибыль выше чем в остальных регионах - 511.36 млн.руб.  
